In [2]:
import json
import re
from embed_llm import DATA_PATH
from datasets import load_dataset

/home/hippolytepilchen/micromamba/envs/pooled_embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Evaluation dataset

In [ ]:
# SQUAD
new_data = []
data = load_dataset("squad", split="validation")
for sample in data:
    new_data.append({
        'question': sample['question'],
        'answer': sample['answers']['text'][0],
        'passages':sample['context']
    })
with open(DATA_PATH + "raw/squad_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

In [ ]:
# CNN/Dailymail
new_data = []
data = load_dataset("abisee/cnn_dailymail","1.0.0", split="test")
for sample in data:
    new_data.append({
        'question': "What is a very short summary of the above text?",
        'answer': sample['highlights'],
        'passages':sample['article']
    })
with open(DATA_PATH + "raw/cnn_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

In [ ]:
#NQ 
new_data = []
data = load_dataset("nq_open",split='validation')
for sample in data:
    new_data.append({
        'question': sample['question'],
        'answer': sample['answer'][0],
    })
with open(DATA_PATH + "raw/nq_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
    

In [ ]:
#TRIVIAQA
new_data = []
data = load_dataset("mandarjoshi/trivia_qa", 'unfiltered.nocontext',split='validation')
for sample in data:
    new_data.append({
        'question': sample['question'],
        'answer': sample['answer']['aliases'],
    })
with open(DATA_PATH + "raw/triviaqa_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
    

In [ ]:
#HotpotQA
new_data = []
data = load_dataset("hotpotqa/hotpot_qa", 'distractor',split='validation')
for sample in data:
    new_data.append({
        'question': sample['question'],
        'answer': sample['answer'],
        'passages': [' '.join(sentences) for sentences in sample['context']['sentences']]
    })
with open(DATA_PATH + "raw/hotpotqa_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
    

In [ ]:
# Atlas chunks for eval 
# Follow instructions from https://github.com/facebookresearch/atlas?tab=readme-ov-file#available-data-and-Models-for-download
# Load it at DATA_PATH + 'raw/Atlas_passages_validation.jsonl' with each line having a 'text' key



In [ ]:
# Load Flores evaluation datasets
fleurs = datasets.load_dataset("facebook/flores", "all", split="dev")

with open(DATA_PATH + "flores/eng_Latn.jsonl", "w") as f:
    for item in fleurs['sentence_eng_Latn']:
        f.write(json.dumps(item) + "\n")
        
with open(DATA_PATH + "flores/spa_Latn.jsonl", "w") as f:
    for item in fleurs['sentence_spa_Latn']:
        f.write(json.dumps(item) + "\n")

with open(DATA_PATH + "flores/fra_Latn.jsonl", "w") as f:
    for item in fleurs['sentence_fra_Latn']:
        f.write(json.dumps(item) + "\n")
        
with open(DATA_PATH + "flores/deu_Latn.jsonl", "w") as f:
    for item in fleurs['sentence_deu_Latn']:
        f.write(json.dumps(item) + "\n")
        
with open(DATA_PATH + "flores/dan_Latn.jsonl", "w") as f:
    for item in fleurs['sentence_dan_Latn']:
        f.write(json.dumps(item) + "\n")


In [ ]:
# Load our fine-tuning dataset from huggingface
!wget https://huggingface.co/datasets/kyutai/ARC_finetuning/resolve/main/ARC-Encoder_ft.zip? -P DATA_PATH + 'w_retrieved/' 
!unzip DATA_PATH + 'w_retrieved/ARC-Encoder_ft.zip' -d . DATA_PATH + 'w_retrieved/'

## Other

### Long-Context Eval

In [ ]:
# PG19 
# Follow instructions from https://github.com/google-deepmind/pg19
# Load training set at DATA_PATH + 'raw/pg19.jsonl' with each line having a 'text' key

In [ ]:
# ArXiv

ds = load_dataset("togethercomputer/RedPajama-Data-1T", 'arxiv', split='train')
new_data = []
for sample in data:
    new_data.append({
        'text': sample['text'],
    })
with open(DATA_PATH + 'raw/arxiv.jsonl', "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
    

In [ ]:

# NQA
data = load_dataset("narrativeqa", split = 'validation')
new_data = []
instruction = "You are given a story, which can be either a novel or a movie script, and a question. Answer the question as concisely as you can, using a single phrase if possible.\n\n"
for sample in data:
    new_data.append(
                {
                    "passage": sample["document"]["text"],
                    "answer": [answer["text"] for answer in sample["answers"]],
                    "question": sample["question"]["text"],
                    "instruction": instruction,
                }
            )

with open(DATA_PATH + "long_context/narrativeqa_validation.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        

# QASPER
data = load_dataset("tau/scrolls", "qasper", split = 'validation')
new_data = []
instruction = 'You are given a scientific article and a question. Answer the question as concisely as you can, using a single phrase or sentence if possible. If the question cannot be answered based on the information in the article, write "unanswerable". If the question is a yes/no question, answer "yes", "no", or "unanswerable".\n\n'
for sample in data:
    new_data.append(
                {
            "passage": sample["input"][
                sample["input"].index("\n\n") + 2 :
            ].strip(),
            "answer": sample["output"],
            "question": sample["input"][
                : sample["input"].index("\n\n")
            ].strip(),
            "instruction": instruction,
        }
    )
with open(DATA_PATH + "long_context/qasper_validation.jsonl","w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
        
# GovReport
valid_dataset = load_dataset("ccdv/govreport-summarization", split = 'validation')
new_data = []
instruction = "Instruction: You are given a report by a government agency. Write a one-page summary of the report.\n"

for sample in valid_dataset:
    new_data.append(
        {
            "passage": sample["report"],
            "answer": sample["summary"],
            "instruction": instruction,
        }
    )


with open(DATA_PATH + "long_context/govreport_validation.jsonl","w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
        
# QMSUM
valid_dataset = load_dataset("tau/scrolls", "qmsum", split = 'validation')
new_data = []
instruction = "You are given a meeting transcript and a query containing a question or instruction. Answer the query in one or more sentences.\n\n"

for sample in valid_dataset:
    new_data.append(
                {
                    "passage": sample["input"][
                        sample["input"].index("\n\n") + 2 :
                    ].strip(),
                    "answer": sample["output"],
                    "question": sample["input"].split("\n")[0].strip(),
                    "instruction": instruction,
                }
            )


with open(DATA_PATH + "long_context/qmsum_validation.jsonl","w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

### Fine-tuning sub-datasets 

In [ ]:
# Freebase QA
new_data = []
data = load_dataset("freebase_qa", split="train")
for sample in data:
    new_data.append({
        'question': sample['RawQuestion'],
        'answer': sample['Parses']['Answers'][0]['AnswersName'][0][0]
    })
with open(DATA_PATH + "raw/freebase_qa_train.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

In [ ]:
# MSMARCO
new_data = []
data = load_dataset("ms_marco","v2.1", split="train")
for sample in data:
    if 'No Answer' in sample['answers'][0]:
        continue
    new_data.append({
        'question': sample['query'],
        'answer': sample['answers'][0]
    })
with open(DATA_PATH + "raw/msmarco_train.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

In [ ]:
# DROP
new_data = []
data = load_dataset("drop", split="train")
for sample in data:
    new_data.append({
        'question': sample['question'],
        'answer': sample['answers_spans']['spans'][0],
        'passages':sample['passage']
    })
with open(DATA_PATH + "raw/drop_train.jsonl", "w") as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

In [ ]:
# WIKISUM

ds = load_dataset("d0rj/wikisum")
data = ds['train']
new_data = []
for sample in data:
    if len(sample['article'])<8000:
        new_data.append({
            'passage':sample['article'],
            'question':'What is a summary of the previous text?',
            'answer':sample['summary']
        })
with open(DATA_PATH + 'WikiSum_train.jsonl','w') as f:
    for item in new_data:
        f.write(json.dumps(item) + '\n')

In [ ]:
# DIALOGSUM

ds = load_dataset("knkarthick/dialogsum")
data = ds['train']
new_data = []
for sample in data:
    if len(sample['dialogue'])<8000:
        new_data.append({
            'passage':sample['dialogue'],
            'question':'Write a short summary of the previous dialogue.',
            'answer':sample['summary']
        })
with open(DATA_PATH + 'DialogSum_train.jsonl','w') as f:
    for item in new_data:
        f.write(json.dumps(item) + '\n')
        

In [ ]:
# SAMSUM

ds = load_dataset("knkarthick/samsum")
data = ds['train']
new_data = []
for sample in data:
    if sample['dialogue'] is None:
        continue
    if len(sample['dialogue'])<8000:
        new_data.append({
            'passage':sample['dialogue'],
            'question':'Write a very short summary of the previous dialogue.',
            'answer':sample['summary']
        })
with open(DATA_PATH + 'SamSum_train.jsonl','w') as f:
    for item in new_data:
        f.write(json.dumps(item) + '\n')

In [ ]:
# ParaSCI
ds = load_dataset("HHousen/ParaSCI")

data = ds['train']
new_data = []
for sample in data:
    if len(sample['sentence1'])>100 and len(sample['sentence2'])>100:
        new_data.append({
            'passage':sample['sentence1'],
            'question':'Paraphrase the previous text.',
            'answer':sample['sentence2'],
        })
with open(DATA_PATH + 'ParaSCI_train.jsonl','w') as f:
    for item in new_data:
        f.write(json.dumps(item) + '\n')

In [ ]:
# Training QA
train_qa_data = {}
ds = load_dataset("dmrau/multi_qa")

for sample in ds['train']:
    data_id = re.sub(r"\d+", "", sample["id"])
    if data_id not in ['adversarial_qa', 'freebase_qa', 'sciq', 'msmarco', 'asqa']:
        continue
    
    if data_id == "msmarco" and "No Answer" in  sample['label']:
        continue
    
    if data_id not in train_qa_data:
        train_qa_data[data_id] = []
        
    train_qa_data[data_id].append({
        'question': sample['content'],
        'answer': sample['label']
    })

for k, v in train_qa_data.items():
    with open(DATA_PATH + f"raw/{k}_train.jsonl", "w") as f:
        for item in v:
            f.write(json.dumps(item) + "\n")

adversarial_qa 29966
msmarco 59699
freebase_qa 20356
sciq 11679
asqa 4353


In [ ]:
# KILT passages for training retrieval
ds = load_dataset("dmrau/kilt-128")
new_data = []
for sample in ds['train']:

    new_data.append({
            'content':sample['text']
    })
with open(DATA_PATH + 'KILT_passages_train.jsonl','w') as f:
    for item in new_data:
        f.write(json.dumps(item) + '\n')